Perform EDA and data cleaning 

In [2]:
import pandas as pd

In [3]:
links =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/links.csv")
links.head()


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
dtypes: int64(1), object(2)
memory usage: 228.4+ KB


In [9]:
ratings =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/ratings.csv")
ratings_final = ratings.drop(columns= "timestamp")
ratings_final.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [48]:
df2 = pd.merge(ratings, movies, how='left' , on='movieId')
df2.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [49]:
tags =  pd.read_csv("/Users/soledadmusella/Desktop/data project 4/tags.csv")
tags.head()


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [50]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
userId       3683 non-null int64
movieId      3683 non-null int64
tag          3683 non-null object
timestamp    3683 non-null int64
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [51]:
df3 = pd.merge(df2, tags, how='left' , on= ["userId",'movieId'],suffixes=("_ratings","_tags"))
df3.head()

,userId,movieId,rating,timestamp_ratings,title,genres,tag,timestamp_tags
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,NaN,NaN
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,NaN,NaN
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,NaN,NaN


In [52]:
df_nonull = df3[~df3.tag.isnull()].reset_index() #df no nulls 
df_nonull.head()

,index,userId,movieId,rating,timestamp_ratings,title,genres,tag,timestamp_tags
0,241,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,funny,1.445715e+09
1,242,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,Highly quotable,1.445715e+09
2,243,2,60756,5.0,1445714980,Step Brothers (2008),Comedy,will ferrell,1.445715e+09
3,252,2,89774,5.0,1445715189,Warrior (2011),Drama,Boxing story,1.445715e+09
4,253,2,89774,5.0,1445715189,Warrior (2011),Drama,MMA,1.445715e+09


In [53]:
final_df = df3.drop(columns=['timestamp_tags',"timestamp_ratings","genres","title","tag"])
final_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


Collaborative filtering 

In [18]:
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
reader = Reader()
df_fin = Dataset.load_from_df(ratings_final,reader)

In [20]:
trainset, testset = train_test_split(df_fin, test_size=0.2)

In [21]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [22]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  610 

Number of items:  9004 



 For the sake of computation time, it's best to calculate the similarity between whichever number is fewer

In [14]:
sim_cos = {'name':'cosine', 'user_based':True}

In [23]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [24]:
basic.sim

array([[1.        , 0.96371587, 0.93900265, ..., 0.96922018, 0.93049713,
        0.77036068],
       [0.96371587, 1.        , 0.94333586, ..., 0.99892863, 0.95940322,
        0.99778516],
       [0.93900265, 0.94333586, 1.        , ..., 0.92384832, 0.9354357 ,
        0.99429638],
       ...,
       [0.96922018, 0.99892863, 0.92384832, ..., 1.        , 1.        ,
        0.        ],
       [0.93049713, 0.95940322, 0.9354357 , ..., 1.        , 1.        ,
        0.        ],
       [0.77036068, 0.99778516, 0.99429638, ..., 0.        , 0.        ,
        1.        ]])

In [25]:
predictions = basic.test(testset)

In [26]:
print(accuracy.rmse(predictions))

RMSE: 0.9731
0.9730945334790014


In [28]:
#looks really good model

In [30]:
sim_pearson = {'name':'pearson', 'user_based':True}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9712
0.9712290808615174


KNN wiht Means takes into account the mean rating of each user or item depending on performing user-user or item-item 

In [32]:
sim_pearson = {'name':'pearson', 'user_based':True}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8945
0.8945292844272862


In [33]:
# this model is giving a better RMSE

In [34]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

In [35]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.8880
0.8879836223108031


In [38]:
#matrix factorization performed better

In [39]:
user_34_prediction = svd.predict('50', '30')
user_34_prediction

Prediction(uid='50', iid='30', r_ui=None, est=3.5032540784449844, details={'was_impossible': False})

In [40]:
user_34_prediction[3]

3.5032540784449844

## OBTAIN USER RATINGS

In [25]:
def movie_rater(movies,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movies[movies['genres'].str.contains(genre)].sample(1)
        else:
            movie = movies.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list  